In [1]:
# Standard library imports
import re

# Third-party library imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, TargetEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    mean_squared_error,
    roc_auc_score,
    recall_score,
    balanced_accuracy_score,
)
from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show
from sklearn.utils import Bunch


In [2]:
# Define the chunk size for reading the CSV file
chunksize = 100000  # Adjust this value based on your requirements

# Initialize an empty list to store filtered chunks
filtered_chunks = []

# Read the CSV file in chunks based on the defined chunk size
for chunk in pd.read_csv("/home/athyrson/Code/Data/Raw Data/Lending Club/accepted_2007_to_2018Q4.csv", chunksize=chunksize):
    # Filter the current chunk based on the criteria
    filtered_chunk = chunk[chunk['issue_d'].str.contains("2010", na=False)]
    # filtered_chunk = filtered_chunk[~filtered_chunk['issue_d'].str.contains("Oct-2013|Nov-2013|Dec-2013", na=False)]
    # Append the filtered chunk to the list
    filtered_chunks.append(filtered_chunk)

# Concatenate all filtered chunks into a single DataFrame
df = pd.concat(filtered_chunks)


/tmp/ipykernel_13044/2038679434.py:8: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("/home/athyrson/Code/Data/Raw Data/Lending Club/accepted_2007_to_2018Q4.csv", chunksize=chunksize):
/tmp/ipykernel_13044/2038679434.py:8: DtypeWarning: Columns (19,59) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("/home/athyrson/Code/Data/Raw Data/Lending Club/accepted_2007_to_2018Q4.csv", chunksize=chunksize):
/tmp/ipykernel_13044/2038679434.py:8: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("/home/athyrson/Code/Data/Raw Data/Lending Club/accepted_2007_to_2018Q4.csv", chunksize=chunksize):
/tmp/ipykernel_13044/2038679434.py:8: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("/home/athyrson/Code/Data/Raw Dat

In [3]:
df_1 = df.sample(frac=1, random_state=42)
df_1.shape

(12537, 151)

In [4]:
import re

def preprocess_data_lc(df):
    # Calculate risk score
    # df["risk_score"] = df.loc[:, ["last_fico_range_high", "last_fico_range_low"]].mean(axis=1)
    
    # Create target variable
    df["target"] = np.where((df.loan_status == 'Current') |
                            (df.loan_status == 'Fully Paid') |
                            (df.loan_status == "Issued") |
                            (df.loan_status == 'Does not meet the credit policy. Status:Fully Paid'), 0, 1)
    
    # Drop unnecessary columns
    # for c in ['last_fico_range_high', 'last_fico_range_low', 'loan_status']:
    for c in ['loan_status']:
        df = df.drop(c, axis=1, errors='ignore')
    
    def fix_term_column(df):
        """
        Fix the dtype of the 'term' column by extracting the numeric part and converting it to an integer.
        Handles cases where the column contains non-string values (e.g., NaN or float).
        
        Parameters:
            df (pd.DataFrame): The DataFrame containing the 'term' column.
        
        Returns:
            pd.DataFrame: The DataFrame with the 'term' column converted to numeric.
        """
        try:
            # Extract numeric part from the 'term' column and convert to integer
            df['term'] = df['term'].apply(lambda x: int(re.search(r'\d+', str(x)).group()) if pd.notna(x) else x)
        except AttributeError as e:
            # Handle cases where the regex search fails (e.g., no match found)
            print("Error: Unable to extract numeric value from 'term' column.")
            print(f"Details: {e}")
        except Exception as e:
            # Handle any other unexpected errors
            print("An unexpected error occurred while fixing the 'term' column.")
            print(f"Details: {e}")
        
        return df
    
    def fix_emp_length_column(df):
        """
        Fix the dtype of the 'emp_length' column by converting it to numeric.
        Handles cases where the column contains non-string values (e.g., NaN or float).
        
        Parameters:
            df (pd.DataFrame): The DataFrame containing the 'emp_length' column.
        
        Returns:
            pd.DataFrame: The DataFrame with the 'emp_length' column converted to numeric.
        """
        try:
            # Map '< 1 year' to '0' and extract numeric part from the 'emp_length' column
            df['emp_length'] = df['emp_length'].map(lambda x: "0" if x == '< 1 year' else x)
            df['emp_length'] = df['emp_length'].map(lambda x: int(re.search(r'\d+', str(x)).group()), na_action='ignore')
        except Exception as e:
            # Handle any unexpected errors
            print("Error: Unable to convert 'emp_length' column to numeric.")
            print(f"Details: {e}")
        
        return df
    
    df = fix_term_column(df)
    df = fix_emp_length_column(df)

    return df

In [5]:
df_lc_1 = preprocess_data_lc(df_1)
df_lc_1.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,target
1643125,498726,NaN,5000.0,5000.0,5000.000000,36,11.36,164.56,B,B5,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0
1636584,597920,NaN,10000.0,10000.0,9861.357548,36,7.88,312.82,A,A5,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0
1639847,543951,NaN,8700.0,8700.0,8700.000000,36,7.51,270.67,A,A4,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0
1643531,492732,NaN,4000.0,4000.0,4000.000000,36,7.88,125.13,A,A5,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0
1640954,528880,NaN,10000.0,10000.0,9362.071323,36,7.14,309.42,A,A3,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0


In [6]:
drop_list = ['member_id',
    'issue_d', 'url', 'collection_recovery_fee', 'last_pymnt_amnt', 'last_pymnt_d', 'pymnt_plan', 'recoveries', 
    'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp', 'hardship_flag', 
    'hardship_type', 'hardship_reason', 'hardship_status', 'deferral_term', 'hardship_amount', 'hardship_start_date', 
    'hardship_end_date', 'payment_plan_start_date', 'hardship_length', 'hardship_dpd', 'hardship_loan_status', 
    'orig_projected_additional_accrued_interest', 'hardship_payoff_balance_amount', 'hardship_last_payment_amount', 
    'debt_settlement_flag', 'debt_settlement_flag_date', 'settlement_status', 'settlement_date', 'settlement_amount', 
    'settlement_percentage', 'settlement_term', 'id', 'emp_title', 'desc', 'title', 'last_credit_pull_d', 'next_pymnt_d',
]

In [7]:
X = df_lc_1.loc[:, df_lc_1.columns != 'target'].drop(drop_list, axis=1)#, 'risk_score'
y = df_lc_1['target']

X_train_lc, X_test_lc, y_train_lc, y_test_lc = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_lc.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,disbursement_method
1644380,25000.0,25000.0,24925.0,36,16.07,879.85,D,D5,10.0,OWN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
1637099,8000.0,8000.0,8000.0,36,7.88,250.25,A,A5,5.0,MORTGAGE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
1638157,6000.0,6000.0,6000.0,36,11.86,198.89,B,B5,0.0,MORTGAGE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
1633647,1200.0,1200.0,1200.0,60,12.23,26.84,C,C1,10.0,MORTGAGE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
1635282,3000.0,3000.0,3000.0,36,8.88,95.24,B,B1,10.0,MORTGAGE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash


In [8]:
df_hc = pd.read_csv("../../Data/Raw Data/HomeCredit/application_train.csv")
df_hc_1 = df_hc.sample(frac=0.3)

In [9]:
X = df_hc_1.loc[:, df_hc_1.columns != 'TARGET'].drop(['SK_ID_CURR'], axis=1)
y = df_hc_1['TARGET']

X_train_hc, X_test_hc, y_train_hc, y_test_hc = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
df_gm = pd.read_csv("/home/athyrson/Code/Data/Raw Data/GiveMeSomeCredit/GMSC.csv")
df_gm.head()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [11]:
X = df_gm.loc[:, df_gm.columns != 'SeriousDlqin2yrs']
y = df_gm['SeriousDlqin2yrs']

X_train_gm, X_test_gm, y_train_gm, y_test_gm = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_gm.head()

,Unnamed: 0,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
104025,104026,0.000000,29,0,0.011513,4342.0,5,0,0,0,0.0
5415,5416,0.595526,55,0,0.835333,1833.0,11,0,1,0,0.0
75612,75613,0.000000,43,0,0.043437,4166.0,2,0,0,0,4.0
138169,138170,0.391980,40,0,0.059771,9000.0,2,0,0,0,5.0
87184,87185,0.000000,35,0,0.133598,5800.0,12,0,1,0,0.0


In [12]:
ebm_lc = ExplainableBoostingClassifier()
ebm_lc.fit(X_train_lc, y_train_lc)


/home/athyrson/Code/py10/lib64/python3.10/site-packages/interpret/utils/_preprocessor.py:358: RuntimeWarning: All-NaN slice encountered
  min_feature_val = np.nanmin(X_col)
/home/athyrson/Code/py10/lib64/python3.10/site-packages/interpret/utils/_preprocessor.py:359: RuntimeWarning: All-NaN slice encountered
  max_feature_val = np.nanmax(X_col)
/home/athyrson/Code/py10/lib64/python3.10/site-packages/interpret/glassbox/_ebm/_ebm.py:813: UserWarning: Missing values detected. Our visualizations do not currently display missing values. To retain the glassbox nature of the model you need to either set the missing values to an extreme value like -1000 that will be visible on the graphs, or manually examine the missing value score in ebm.term_scores_[term_index][0]
  warn(


ExplainableBoostingClassifier()

In [13]:
ebm_hc = ExplainableBoostingClassifier()
ebm_hc.fit(X_train_hc, y_train_hc)

ExplainableBoostingClassifier()

In [14]:
ebm_gm = ExplainableBoostingClassifier()
ebm_gm.fit(X_train_gm, y_train_gm)

ExplainableBoostingClassifier()

In [15]:
#auc
y_pred_lc = ebm_lc.predict_proba(X_test_lc)[:, 1]
y_pred_hc = ebm_hc.predict_proba(X_test_hc)[:, 1]
y_pred_gm = ebm_gm.predict_proba(X_test_gm)[:, 1]

auc_lc = roc_auc_score(y_test_lc, y_pred_lc)
auc_hc = roc_auc_score(y_test_hc, y_pred_hc)
auc_gm = roc_auc_score(y_test_gm, y_pred_gm)

print(f"AUC Lending Club: {auc_lc:.4f}")
print(f"AUC Home Credit: {auc_hc:.4f}")
print(f"AUC Give Me Some Credit: {auc_gm:.4f}")

AUC Lending Club: 0.8451
AUC Home Credit: 0.7537
AUC Give Me Some Credit: 0.8636


In [16]:
from interpret import show
show(ebm_lc.explain_global())

<!-- http://127.0.0.1:7001/139841593943488/ -->

In [17]:
from interpret import show
show(ebm_hc.explain_global())

<!-- http://127.0.0.1:7001/139841593937488/ -->

In [18]:
from interpret import show
show(ebm_gm.explain_global())

<!-- http://127.0.0.1:7001/139841489432256/ -->